# TATA Online Retail Store: Revenue Drivers Analysis

---

### Business Studies with Applied AI | Group 11

| # | Member |
|---|--------|
| 1 | **Payal Kunwar** |
| 2 | **Gaurav Kulkarni** |
| 3 | **Anshuman Atrey** |
| 4 | **Abdullah Haque** |
| 5 | **Shlok Vijay Kadam** |

---

## Business Problem Statement

TATA's online retail division generates revenue from customers across **multiple countries**, selling a wide variety of products. However, leadership lacks a **data-driven understanding** of the key factors that drive revenue growth and customer retention. Without this insight, strategic decisions around marketing spend, inventory allocation, geographic expansion, and customer engagement remain sub-optimal.

## Objective

This project applies **Artificial Intelligence and Machine Learning techniques** to the TATA Online Retail transaction dataset to:

1. **Identify and quantify** the primary drivers of revenue (products, countries, time patterns, customer segments).
2. **Segment customers** using RFM analysis and K-Means clustering to enable targeted marketing strategies.
3. **Predict high-value customers** using Logistic Regression to support proactive retention efforts.
4. **Model revenue trends** using Linear Regression to support demand forecasting.
5. **Translate analytical findings** into actionable business recommendations grounded in economic theory.

---

## 1. Import Libraries

We begin by importing all necessary Python libraries for data manipulation, visualization, and machine learning.

In [ ]:
# ============================================================
# 1. IMPORT LIBRARIES
# ============================================================

# Core data manipulation
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Static visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, r2_score, mean_absolute_error, mean_squared_error

# Model persistence
import joblib

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
sns.set_style('whitegrid')

print('All libraries imported successfully.')

## 2. Load Dataset

The dataset is the **UCI Online Retail Dataset**, containing transactional data for a UK-based online retail company between **December 2010 and December 2011**. Each row represents a line item from an invoice.

In [ ]:
# ============================================================
# 2. LOAD DATASET
# ============================================================

df = pd.read_csv('dataset/Online Retail Data Set.csv', encoding='latin1')

print(f'Dataset Shape: {df.shape[0]:,} rows x {df.shape[1]} columns')
print(f'Memory Usage: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB')
print('='*60)
df.head(10)

In [ ]:
# Dataset info and structure
print('DATASET INFO')
print('='*60)
df.info()
print('\n')
print('DESCRIPTIVE STATISTICS')
print('='*60)
df.describe()

In [ ]:
# Check for missing values and data quality
print('MISSING VALUES SUMMARY')
print('='*60)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0])
print(f'\nTotal rows with null CustomerID: {df["CustomerID"].isnull().sum():,}')
print(f'Cancellation invoices (starting with C): {df["InvoiceNo"].astype(str).str.startswith("C").sum():,}')
print(f'Negative Quantity rows: {(df["Quantity"] < 0).sum():,}')
print(f'Negative/Zero UnitPrice rows: {(df["UnitPrice"] <= 0).sum():,}')
print(f'Unique Countries: {df["Country"].nunique()}')
print(f'Unique Customers: {df["CustomerID"].nunique()}')
print(f'Unique Products: {df["StockCode"].nunique()}')

## 3. Data Cleaning & Preprocessing

Raw transactional data contains noise that can distort our analysis. We apply the following cleaning steps:

| Step | Action | Rationale |
|------|--------|-----------|
| 1 | Remove cancelled orders (InvoiceNo starts with 'C') | Cancellations do not represent actual revenue |
| 2 | Remove rows with Quantity <= 0 | Negative/zero quantities are returns or data errors |
| 3 | Remove rows with UnitPrice <= 0 | Zero or negative prices are likely adjustments, not real sales |
| 4 | Drop null CustomerID rows | Required for customer-level analysis (RFM, clustering) |
| 5 | Create TotalRevenue column | Revenue = Quantity x UnitPrice |
| 6 | Parse dates and extract time features | Enable temporal trend analysis |

In [ ]:
# ============================================================
# 3. DATA CLEANING & PREPROCESSING
# ============================================================

shape_before = df.shape
print(f'Shape BEFORE cleaning: {shape_before[0]:,} rows x {shape_before[1]} columns')
print('-'*60)

# Step 1: Remove cancelled orders (InvoiceNo starting with 'C')
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
cancelled = df['InvoiceNo'].str.startswith('C').sum()
df = df[~df['InvoiceNo'].str.startswith('C')]
print(f'Step 1 - Removed {cancelled:,} cancelled orders')

# Step 2: Remove rows with Quantity <= 0
neg_qty = (df['Quantity'] <= 0).sum()
df = df[df['Quantity'] > 0]
print(f'Step 2 - Removed {neg_qty:,} rows with Quantity <= 0')

# Step 3: Remove rows with UnitPrice <= 0
neg_price = (df['UnitPrice'] <= 0).sum()
df = df[df['UnitPrice'] > 0]
print(f'Step 3 - Removed {neg_price:,} rows with UnitPrice <= 0')

# Step 4: Drop rows with null CustomerID
null_cust = df['CustomerID'].isnull().sum()
df = df.dropna(subset=['CustomerID'])
df['CustomerID'] = df['CustomerID'].astype(int)
print(f'Step 4 - Removed {null_cust:,} rows with null CustomerID')

# Step 5: Create TotalRevenue column
df['TotalRevenue'] = df['Quantity'] * df['UnitPrice']
print(f'Step 5 - Created TotalRevenue column')

# Step 6: Parse InvoiceDate and extract time features
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='mixed', dayfirst=True)
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Hour'] = df['InvoiceDate'].dt.hour
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # 0=Monday, 6=Sunday
df['DayName'] = df['InvoiceDate'].dt.day_name()
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')
print(f'Step 6 - Parsed dates and extracted Month, Year, Hour, DayOfWeek')

print('-'*60)
shape_after = df.shape
print(f'Shape AFTER cleaning: {shape_after[0]:,} rows x {shape_after[1]} columns')
print(f'Rows removed: {shape_before[0] - shape_after[0]:,} ({(shape_before[0] - shape_after[0])/shape_before[0]*100:.1f}%)')
print(f'\nTotal Revenue in cleaned dataset: GBP {df["TotalRevenue"].sum():,.2f}')

In [ ]:
# Summary statistics after cleaning
print('CLEANED DATASET - SUMMARY STATISTICS')
print('='*60)
df.describe()

In [ ]:
# Quick look at the cleaned data
df.head()

---

## 4. Exploratory Data Analysis (EDA)

With clean data in hand, we now explore the key dimensions of revenue:
- **Temporal patterns** (monthly trends, hourly patterns, day-of-week effects)
- **Geographic distribution** (revenue by country)
- **Product analysis** (top-selling items)
- **Customer behavior** (purchase frequency)
- **Correlations** between numerical features

---

### 4.1 Monthly Revenue Trend

In [ ]:
# ============================================================
# 4.1 MONTHLY REVENUE TREND
# ============================================================

monthly_revenue = df.groupby('YearMonth').agg(
    Revenue=('TotalRevenue', 'sum'),
    Orders=('InvoiceNo', 'nunique'),
    Customers=('CustomerID', 'nunique')
).reset_index()
monthly_revenue['YearMonth'] = monthly_revenue['YearMonth'].astype(str)

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Monthly Revenue (GBP)', 'Monthly Unique Orders & Customers'),
    vertical_spacing=0.12
)

# Revenue line
fig.add_trace(
    go.Scatter(
        x=monthly_revenue['YearMonth'],
        y=monthly_revenue['Revenue'],
        mode='lines+markers',
        name='Revenue (GBP)',
        line=dict(color='#2ecc71', width=3),
        marker=dict(size=8),
        fill='tozeroy',
        fillcolor='rgba(46, 204, 113, 0.15)'
    ),
    row=1, col=1
)

# Orders and customers
fig.add_trace(
    go.Scatter(
        x=monthly_revenue['YearMonth'],
        y=monthly_revenue['Orders'],
        mode='lines+markers',
        name='Unique Orders',
        line=dict(color='#3498db', width=2)
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=monthly_revenue['YearMonth'],
        y=monthly_revenue['Customers'],
        mode='lines+markers',
        name='Unique Customers',
        line=dict(color='#e74c3c', width=2)
    ),
    row=2, col=1
)

fig.update_layout(
    height=700,
    title_text='Revenue & Customer Activity Over Time',
    title_font_size=20,
    template='plotly_dark',
    showlegend=True
)
fig.update_yaxes(title_text='Revenue (GBP)', row=1, col=1)
fig.update_yaxes(title_text='Count', row=2, col=1)
fig.show()

### 4.2 Top 10 Revenue-Generating Countries (Excluding UK)

In [ ]:
# ============================================================
# 4.2 TOP 10 REVENUE-GENERATING COUNTRIES (Excluding UK)
# ============================================================

country_revenue = df.groupby('Country')['TotalRevenue'].sum().sort_values(ascending=False).reset_index()

# Show UK dominance first
uk_rev = country_revenue[country_revenue['Country'] == 'United Kingdom']['TotalRevenue'].values[0]
total_rev = country_revenue['TotalRevenue'].sum()
print(f'UK Revenue: GBP {uk_rev:,.2f} ({uk_rev/total_rev*100:.1f}% of total)')
print(f'Non-UK Revenue: GBP {total_rev - uk_rev:,.2f} ({(total_rev-uk_rev)/total_rev*100:.1f}% of total)')

# Plot top 10 excluding UK for better visibility
top10_no_uk = country_revenue[country_revenue['Country'] != 'United Kingdom'].head(10)

fig = px.bar(
    top10_no_uk,
    x='TotalRevenue',
    y='Country',
    orientation='h',
    color='TotalRevenue',
    color_continuous_scale='Viridis',
    title='Top 10 Revenue-Generating Countries (Excluding UK)',
    labels={'TotalRevenue': 'Total Revenue (GBP)', 'Country': ''},
    text=top10_no_uk['TotalRevenue'].apply(lambda x: f'GBP {x:,.0f}')
)
fig.update_layout(
    template='plotly_dark',
    height=500,
    title_font_size=18,
    yaxis=dict(autorange='reversed')
)
fig.update_traces(textposition='outside')
fig.show()

### 4.3 Top 20 Best-Selling Products by Revenue

In [ ]:
# ============================================================
# 4.3 TOP 20 BEST-SELLING PRODUCTS BY REVENUE
# ============================================================

product_revenue = df.groupby(['StockCode', 'Description']).agg(
    Revenue=('TotalRevenue', 'sum'),
    Quantity=('Quantity', 'sum'),
    Orders=('InvoiceNo', 'nunique')
).sort_values('Revenue', ascending=False).head(20).reset_index()

# Truncate long descriptions for display
product_revenue['ShortDesc'] = product_revenue['Description'].str[:40]

fig = px.bar(
    product_revenue,
    x='Revenue',
    y='ShortDesc',
    orientation='h',
    color='Revenue',
    color_continuous_scale='Turbo',
    title='Top 20 Best-Selling Products by Revenue',
    labels={'Revenue': 'Total Revenue (GBP)', 'ShortDesc': 'Product'},
    hover_data={'Description': True, 'Quantity': True, 'Orders': True}
)
fig.update_layout(
    template='plotly_dark',
    height=700,
    title_font_size=18,
    yaxis=dict(autorange='reversed')
)
fig.show()

### 4.4 Revenue Distribution by Day of Week

In [ ]:
# ============================================================
# 4.4 REVENUE DISTRIBUTION BY DAY OF WEEK
# ============================================================

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_revenue = df.groupby('DayName').agg(
    Revenue=('TotalRevenue', 'sum'),
    AvgOrderValue=('TotalRevenue', 'mean'),
    Orders=('InvoiceNo', 'nunique')
).reindex(day_order).reset_index()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Total Revenue by Day of Week', 'Average Order Value by Day of Week')
)

colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c', '#e67e22']

fig.add_trace(
    go.Bar(
        x=daily_revenue['DayName'],
        y=daily_revenue['Revenue'],
        marker_color=colors,
        name='Total Revenue',
        text=daily_revenue['Revenue'].apply(lambda x: f'GBP {x:,.0f}'),
        textposition='outside'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=daily_revenue['DayName'],
        y=daily_revenue['AvgOrderValue'],
        marker_color=colors,
        name='Avg Order Value',
        text=daily_revenue['AvgOrderValue'].apply(lambda x: f'GBP {x:.2f}'),
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    template='plotly_dark',
    height=500,
    title_text='Revenue Patterns by Day of Week',
    title_font_size=18,
    showlegend=False
)
fig.show()

### 4.5 Hourly Sales Pattern

In [ ]:
# ============================================================
# 4.5 HOURLY SALES PATTERN
# ============================================================

hourly_revenue = df.groupby('Hour').agg(
    Revenue=('TotalRevenue', 'sum'),
    Transactions=('InvoiceNo', 'nunique')
).reset_index()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=hourly_revenue['Hour'],
        y=hourly_revenue['Revenue'],
        mode='lines+markers',
        name='Revenue (GBP)',
        line=dict(color='#2ecc71', width=3),
        marker=dict(size=8),
        fill='tozeroy',
        fillcolor='rgba(46, 204, 113, 0.1)'
    ),
    secondary_y=False
)

fig.add_trace(
    go.Bar(
        x=hourly_revenue['Hour'],
        y=hourly_revenue['Transactions'],
        name='Unique Transactions',
        marker_color='rgba(52, 152, 219, 0.5)',
        opacity=0.6
    ),
    secondary_y=True
)

fig.update_layout(
    title='Hourly Sales Pattern: Revenue & Transaction Volume',
    title_font_size=18,
    template='plotly_dark',
    height=500,
    xaxis_title='Hour of Day',
    xaxis=dict(dtick=1)
)
fig.update_yaxes(title_text='Revenue (GBP)', secondary_y=False)
fig.update_yaxes(title_text='Unique Transactions', secondary_y=True)
fig.show()

### 4.6 Revenue by Country - World Map (Choropleth)

In [ ]:
# ============================================================
# 4.6 REVENUE BY COUNTRY - WORLD MAP (CHOROPLETH)
# ============================================================

# Map country names to ISO codes for plotly choropleth
country_iso = {
    'United Kingdom': 'GBR', 'France': 'FRA', 'Australia': 'AUS', 'Netherlands': 'NLD',
    'Germany': 'DEU', 'Norway': 'NOR', 'EIRE': 'IRL', 'Switzerland': 'CHE',
    'Spain': 'ESP', 'Poland': 'POL', 'Portugal': 'PRT', 'Italy': 'ITA',
    'Belgium': 'BEL', 'Lithuania': 'LTU', 'Japan': 'JPN', 'Iceland': 'ISL',
    'Channel Islands': 'GBR', 'Denmark': 'DNK', 'Cyprus': 'CYP', 'Sweden': 'SWE',
    'Austria': 'AUT', 'Israel': 'ISR', 'Finland': 'FIN', 'Bahrain': 'BHR',
    'Greece': 'GRC', 'Hong Kong': 'HKG', 'Singapore': 'SGP', 'Lebanon': 'LBN',
    'United Arab Emirates': 'ARE', 'Saudi Arabia': 'SAU', 'Czech Republic': 'CZE',
    'Canada': 'CAN', 'Unspecified': None, 'Brazil': 'BRA', 'USA': 'USA',
    'European Community': None, 'Malta': 'MLT', 'RSA': 'ZAF'
}

country_rev = df.groupby('Country')['TotalRevenue'].sum().reset_index()
country_rev['ISO'] = country_rev['Country'].map(country_iso)
country_rev = country_rev.dropna(subset=['ISO'])

fig = px.choropleth(
    country_rev,
    locations='ISO',
    color='TotalRevenue',
    hover_name='Country',
    color_continuous_scale='Plasma',
    title='Global Revenue Distribution by Country',
    labels={'TotalRevenue': 'Revenue (GBP)'},
    projection='natural earth'
)
fig.update_layout(
    template='plotly_dark',
    height=600,
    title_font_size=18,
    geo=dict(showframe=False, showcoastlines=True, coastlinecolor='gray')
)
fig.show()

### 4.7 Customer Purchase Frequency Distribution

In [ ]:
# ============================================================
# 4.7 CUSTOMER PURCHASE FREQUENCY DISTRIBUTION
# ============================================================

customer_freq = df.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
customer_freq.columns = ['CustomerID', 'PurchaseFrequency']

fig = px.histogram(
    customer_freq,
    x='PurchaseFrequency',
    nbins=50,
    title='Customer Purchase Frequency Distribution',
    labels={'PurchaseFrequency': 'Number of Purchases', 'count': 'Number of Customers'},
    color_discrete_sequence=['#3498db']
)
fig.update_layout(
    template='plotly_dark',
    height=500,
    title_font_size=18,
    xaxis_title='Number of Purchases',
    yaxis_title='Number of Customers'
)

# Add annotation for median
median_freq = customer_freq['PurchaseFrequency'].median()
fig.add_vline(x=median_freq, line_dash='dash', line_color='red',
              annotation_text=f'Median: {median_freq:.0f}', annotation_position='top right')
fig.show()

print(f'Purchase Frequency Statistics:')
print(f'  Mean:   {customer_freq["PurchaseFrequency"].mean():.1f} orders/customer')
print(f'  Median: {customer_freq["PurchaseFrequency"].median():.0f} orders/customer')
print(f'  Max:    {customer_freq["PurchaseFrequency"].max()} orders/customer')
print(f'  Customers with only 1 purchase: {(customer_freq["PurchaseFrequency"]==1).sum():,} ({(customer_freq["PurchaseFrequency"]==1).mean()*100:.1f}%)')

### 4.8 Correlation Heatmap

In [ ]:
# ============================================================
# 4.8 CORRELATION HEATMAP
# ============================================================

numeric_cols = ['Quantity', 'UnitPrice', 'TotalRevenue', 'Month', 'Hour', 'DayOfWeek']
corr_matrix = df[numeric_cols].corr()

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Correlation Heatmap of Numerical Features',
    labels=dict(color='Correlation'),
    aspect='auto'
)
fig.update_layout(
    template='plotly_dark',
    height=550,
    title_font_size=18
)
fig.show()

---

## 5. RFM Analysis (Recency, Frequency, Monetary)

RFM analysis is a **customer segmentation technique** based on three behavioral metrics:

| Metric | Definition | Business Meaning |
|--------|-----------|------------------|
| **Recency (R)** | Days since last purchase | How recently did the customer buy? |
| **Frequency (F)** | Number of unique purchases | How often does the customer buy? |
| **Monetary (M)** | Total amount spent | How much does the customer spend? |

Customers who bought **recently**, buy **frequently**, and spend **more** are the most valuable.

---

In [ ]:
# ============================================================
# 5. RFM ANALYSIS
# ============================================================

# Reference date = max date in dataset + 1 day
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f'Reference date for Recency calculation: {reference_date}')

# Calculate RFM per customer
rfm = df.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (reference_date - x.max()).days),
    Frequency=('InvoiceNo', 'nunique'),
    Monetary=('TotalRevenue', 'sum')
).reset_index()

print(f'\nRFM Table Shape: {rfm.shape}')
print(f'Total unique customers analyzed: {rfm.shape[0]:,}')
print('\nRFM Summary Statistics:')
rfm[['Recency', 'Frequency', 'Monetary']].describe()

In [ ]:
# RFM Distribution plots
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Recency (days)', 'Frequency (orders)', 'Monetary (GBP)')
)

fig.add_trace(
    go.Histogram(x=rfm['Recency'], nbinsx=50, marker_color='#e74c3c', name='Recency', opacity=0.8),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=rfm['Frequency'], nbinsx=50, marker_color='#3498db', name='Frequency', opacity=0.8),
    row=1, col=2
)
fig.add_trace(
    go.Histogram(x=rfm['Monetary'], nbinsx=50, marker_color='#2ecc71', name='Monetary', opacity=0.8),
    row=1, col=3
)

fig.update_layout(
    title_text='RFM Distributions',
    title_font_size=18,
    template='plotly_dark',
    height=400,
    showlegend=False
)
fig.show()

In [ ]:
# ============================================================
# 5.1 ASSIGN RFM SCORES (1-4 using quantiles)
# ============================================================

# For Recency: lower is better, so we invert the scoring
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=4, labels=[4, 3, 2, 1]).astype(int)

# For Frequency and Monetary: higher is better
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=4, labels=[1, 2, 3, 4]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=4, labels=[1, 2, 3, 4]).astype(int)

# Combined RFM Score
rfm['RFM_Score'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

print('RFM Scores assigned (1-4 scale, 4 = best).')
print(f'RFM_Score range: {rfm["RFM_Score"].min()} to {rfm["RFM_Score"].max()}')
rfm.head(10)

In [ ]:
# ============================================================
# 5.2 CREATE CUSTOMER SEGMENTS
# ============================================================

def assign_segment(row):
    """Assign customer segment based on RFM scores."""
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']

    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Loyal Customers'
    elif r >= 3 and f >= 2 and m >= 2:
        return 'Potential Loyalists'
    elif r >= 4 and f <= 2:
        return 'New Customers'
    elif r >= 2 and f >= 2 and m >= 2:
        return 'Promising'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    elif r <= 2 and f >= 4 and m >= 4:
        return 'Cannot Lose Them'
    elif r <= 2 and f <= 2 and m <= 2:
        return 'Lost'
    elif r <= 2 and f <= 2:
        return 'Hibernating'
    else:
        return 'Need Attention'

rfm['Segment'] = rfm.apply(assign_segment, axis=1)

# Segment summary
segment_summary = rfm.groupby('Segment').agg(
    Count=('CustomerID', 'count'),
    Avg_Recency=('Recency', 'mean'),
    Avg_Frequency=('Frequency', 'mean'),
    Avg_Monetary=('Monetary', 'mean'),
    Total_Revenue=('Monetary', 'sum')
).sort_values('Total_Revenue', ascending=False).round(1)

segment_summary['Revenue_Pct'] = (segment_summary['Total_Revenue'] / segment_summary['Total_Revenue'].sum() * 100).round(1)
print('CUSTOMER SEGMENT SUMMARY')
print('='*80)
segment_summary

In [ ]:
# Segment Visualization - Pie Chart & Bar Chart
segment_counts = rfm['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']

segment_rev = rfm.groupby('Segment')['Monetary'].sum().sort_values(ascending=True).reset_index()

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "bar"}]],
    subplot_titles=('Customer Distribution by Segment', 'Total Revenue by Segment')
)

colors_seg = px.colors.qualitative.Set2

fig.add_trace(
    go.Pie(
        labels=segment_counts['Segment'],
        values=segment_counts['Count'],
        hole=0.4,
        marker_colors=colors_seg,
        textinfo='percent+label',
        textfont_size=10
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=segment_rev['Monetary'],
        y=segment_rev['Segment'],
        orientation='h',
        marker_color=colors_seg[:len(segment_rev)],
        text=segment_rev['Monetary'].apply(lambda x: f'GBP {x:,.0f}'),
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='RFM Customer Segmentation Results',
    title_font_size=20,
    template='plotly_dark',
    height=550,
    showlegend=False
)
fig.show()

---

## 6. K-Means Clustering

We apply **K-Means**, an unsupervised machine learning algorithm, to cluster customers based on their **RFM values**. Unlike the rule-based RFM segmentation above, K-Means finds natural groupings in the data without predefined boundaries.

**Steps:**
1. Standardize RFM values (zero mean, unit variance) to prevent scale bias.
2. Use the **Elbow Method** and **Silhouette Scores** to determine the optimal number of clusters.
3. Fit K-Means and analyze each cluster's business profile.

---

In [ ]:
# ============================================================
# 6. K-MEANS CLUSTERING
# ============================================================

# Prepare features for clustering
rfm_features = rfm[['Recency', 'Frequency', 'Monetary']].copy()

# Standardize
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_features)

print('Features standardized using StandardScaler.')
print(f'Scaled data shape: {rfm_scaled.shape}')

In [ ]:
# ============================================================
# 6.1 ELBOW METHOD + SILHOUETTE ANALYSIS
# ============================================================

K_range = range(2, 11)
inertias = []
silhouette_scores = []

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    labels_temp = kmeans_temp.fit_predict(rfm_scaled)
    inertias.append(kmeans_temp.inertia_)
    silhouette_scores.append(silhouette_score(rfm_scaled, labels_temp))
    print(f'  k={k}: Inertia={kmeans_temp.inertia_:,.0f}, Silhouette={silhouette_scores[-1]:.4f}')

# Plot Elbow and Silhouette side by side
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Elbow Method (Inertia vs K)', 'Silhouette Score vs K')
)

fig.add_trace(
    go.Scatter(
        x=list(K_range), y=inertias,
        mode='lines+markers',
        line=dict(color='#e74c3c', width=3),
        marker=dict(size=10),
        name='Inertia'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=list(K_range), y=silhouette_scores,
        mode='lines+markers',
        line=dict(color='#2ecc71', width=3),
        marker=dict(size=10),
        name='Silhouette'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text='Number of Clusters (K)', dtick=1, row=1, col=1)
fig.update_xaxes(title_text='Number of Clusters (K)', dtick=1, row=1, col=2)
fig.update_yaxes(title_text='Inertia', row=1, col=1)
fig.update_yaxes(title_text='Silhouette Score', row=1, col=2)

fig.update_layout(
    title_text='Determining Optimal Number of Clusters',
    title_font_size=18,
    template='plotly_dark',
    height=450,
    showlegend=False
)
fig.show()

# Select optimal k
optimal_k = 4
print(f'\nOptimal K selected: {optimal_k} (based on elbow point and silhouette balance)')

In [ ]:

# ============================================================
# 6.2 FIT K-MEANS WITH OPTIMAL K
# ============================================================

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10, max_iter=300)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

print(f'K-Means fitted with K={optimal_k}')
print(f'Final Inertia: {kmeans.inertia_:,.2f}')
print(f'Silhouette Score: {silhouette_score(rfm_scaled, rfm["Cluster"]):.4f}')

# ── Re-map clusters so Cluster 0 = highest avg Monetary (matches utils/data.py logic) ──
order = rfm.groupby('Cluster')['Monetary'].mean().sort_values(ascending=False).index.tolist()
cluster_map = {old: new for new, old in enumerate(order)}
rfm['Cluster'] = rfm['Cluster'].map(cluster_map)
print(f'\nCluster re-mapping (0 = highest value): {cluster_map}')

# ── Assign human-readable names (aligned with Streamlit app config) ──
CLUSTER_NAMES = {0: 'Champions', 1: 'Potential Loyalists', 2: 'At-Risk', 3: 'Hibernating'}
CLUSTER_ACCENT = {0: '#16a34a', 1: '#2563eb', 2: '#d97706', 3: '#dc2626'}
rfm['Segment_ML'] = rfm['Cluster'].map(CLUSTER_NAMES)

print(f'\nCluster Distribution (after re-mapping):')
print(rfm['Cluster'].value_counts().sort_index())
print(f'\nSegment Distribution:')
print(rfm['Segment_ML'].value_counts())


In [ ]:

# ============================================================
# 6.3 3D SCATTER PLOT OF CLUSTERS
# ============================================================

sample = rfm.sample(n=min(3000, len(rfm)), random_state=42)

fig = px.scatter_3d(
    sample,
    x='Recency',
    y='Frequency',
    z='Monetary',
    color='Segment_ML',
    color_discrete_map={v: CLUSTER_ACCENT[k] for k, v in CLUSTER_NAMES.items()},
    title='K-Means Customer Clusters (3D View)',
    labels={'Recency': 'Recency (days)', 'Frequency': 'Frequency (orders)', 'Monetary': 'Monetary (GBP)'},
    opacity=0.65,
    hover_data={'CustomerID': True, 'Recency': True, 'Frequency': True, 'Monetary': ':.2f'}
)

fig.update_layout(
    template='plotly_white',
    height=700,
    title_font_size=18,
    scene=dict(
        xaxis_title='Recency (days)',
        yaxis_title='Frequency (orders)',
        zaxis_title='Monetary (GBP)'
    ),
    legend=dict(title='Segment', font=dict(size=11))
)
fig.show()


In [ ]:

# ============================================================
# 6.4 CLUSTER PROFILE ANALYSIS
# ============================================================

cluster_profile = rfm.groupby('Cluster').agg(
    Segment=('Segment_ML', 'first'),
    Count=('CustomerID', 'count'),
    Avg_Recency=('Recency', 'mean'),
    Avg_Frequency=('Frequency', 'mean'),
    Avg_Monetary=('Monetary', 'mean'),
    Total_Revenue=('Monetary', 'sum')
).round(2)

cluster_profile['Revenue_Share_%'] = (cluster_profile['Total_Revenue'] / cluster_profile['Total_Revenue'].sum() * 100).round(1)
cluster_profile['Customer_Share_%'] = (cluster_profile['Count'] / cluster_profile['Count'].sum() * 100).round(1)

print('CLUSTER PROFILES (aligned with Streamlit app)')
print('='*90)
cluster_profile


In [ ]:

# Visualize cluster profiles with grouped bar chart
cp = cluster_profile.reset_index()
cp['Label'] = cp['Cluster'].astype(str) + ' – ' + cp['Segment']
cluster_colors = [CLUSTER_ACCENT[i] for i in cp['Cluster']]

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Avg Recency (days) ↓ better', 'Avg Frequency (orders) ↑ better', 'Avg Monetary (GBP) ↑ better')
)

fig.add_trace(
    go.Bar(x=cp['Label'], y=cp['Avg_Recency'], marker_color=cluster_colors,
           text=cp['Avg_Recency'].round(0), textposition='outside', name='Recency'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=cp['Label'], y=cp['Avg_Frequency'], marker_color=cluster_colors,
           text=cp['Avg_Frequency'].round(1), textposition='outside', name='Frequency'),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=cp['Label'], y=cp['Avg_Monetary'], marker_color=cluster_colors,
           text=cp['Avg_Monetary'].apply(lambda x: f'{x:,.0f}'), textposition='outside', name='Monetary'),
    row=1, col=3
)

fig.update_layout(
    title_text='Cluster Profiles: Average RFM Values by Segment',
    title_font_size=18,
    template='plotly_white',
    height=450,
    showlegend=False
)
fig.update_xaxes(tickangle=-20)
fig.show()



#### Business Interpretation of K-Means Clusters

Clusters are re-mapped so **Cluster 0 always has the highest average Monetary** (matches deployed app). The segment names are identical to those shown in the Streamlit dashboard.

| Cluster | Segment Name | Recency | Frequency | Monetary | Business Strategy |
|---------|--------------|---------|-----------|----------|-------------------|
| **0** | **Champions** | Low (recent) | High | High | VIP loyalty rewards, early access to launches, referral incentives |
| **1** | **Potential Loyalists** | Moderate | Moderate | Moderate | Upsell & cross-sell, tiered loyalty enrollment, free shipping |
| **2** | **At-Risk** | High (lapsed) | Was High | Was High | Win-back emails, comeback offers, disengagement survey |
| **3** | **Hibernating** | High (dormant) | Low | Low | Aggressive reactivation discount, low-cost trial offers |

> **App alignment:** This mapping is generated by `utils/data.py → train_models()`, which sorts raw KMeans cluster IDs by descending mean Monetary, then assigns labels via `CLUSTER_NAMES` in `utils/config.py`.


---

## 7. Logistic Regression: Predicting High-Value Customers

We build a **supervised classification model** to predict whether a customer is **High-Value** (Monetary > median) or **Low-Value** (Monetary <= median) based on their Recency and Frequency.

**Business Application:** This model can be used to identify which new or returning customers are likely to become high-value, enabling proactive marketing and retention strategies.

---

In [ ]:

# ============================================================
# 7. LOGISTIC REGRESSION
# ============================================================
# NOTE: We use the same StandardScaler + same 3 features (Recency, Frequency, Monetary)
# as KMeans — this is exactly how utils/data.py → train_models() works in the app.

# Create binary target: High-Value = 1 if Monetary > median, else 0
monetary_median = rfm['Monetary'].median()
rfm['HighValue'] = (rfm['Monetary'] > monetary_median).astype(int)

print(f'Monetary Median (threshold): GBP {monetary_median:,.2f}')
print(f'High-Value customers: {rfm["HighValue"].sum():,} ({rfm["HighValue"].mean()*100:.1f}%)')
print(f'Low-Value customers:  {(rfm["HighValue"]==0).sum():,} ({(1-rfm["HighValue"].mean())*100:.1f}%)')

# ── Use the already-scaled RFM features from Section 6 (matches deployed app) ──
# rfm_scaled contains scaled [Recency, Frequency, Monetary]
X = rfm_scaled
y = rfm['HighValue'].values

# Train-test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'\nFeatures used: Scaled [Recency, Frequency, Monetary] — identical to KMeans input')
print(f'  (Aligns with utils/data.py → train_models() in the Streamlit app)')
print(f'\nTrain size: {len(X_train):,}')
print(f'Test size:  {len(X_test):,}')


In [ ]:
# Fit Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)

# Predictions
y_pred = log_reg.predict(X_test)
y_prob = log_reg.predict_proba(X_test)[:, 1]

# Classification Report
print('CLASSIFICATION REPORT')
print('='*60)
print(classification_report(y_test, y_pred, target_names=['Low-Value', 'High-Value']))

In [ ]:
# ============================================================
# 7.1 CONFUSION MATRIX HEATMAP
# ============================================================

cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Low-Value', 'High-Value'],
    yticklabels=['Low-Value', 'High-Value'],
    linewidths=1, linecolor='white',
    annot_kws={'size': 16}
)
ax.set_xlabel('Predicted Label', fontsize=14)
ax.set_ylabel('True Label', fontsize=14)
ax.set_title('Confusion Matrix - Logistic Regression', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

accuracy = (cm[0,0] + cm[1,1]) / cm.sum()
print(f'Accuracy: {accuracy*100:.1f}%')

In [ ]:
# ============================================================
# 7.2 ROC CURVE WITH AUC SCORE
# ============================================================

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'Logistic Regression (AUC = {roc_auc:.3f})',
    line=dict(color='#2ecc71', width=3)
))

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random Classifier (AUC = 0.5)',
    line=dict(color='gray', width=2, dash='dash')
))

fig.update_layout(
    title=f'ROC Curve - Logistic Regression (AUC = {roc_auc:.3f})',
    title_font_size=18,
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_dark',
    height=500,
    legend=dict(x=0.5, y=0.1)
)
fig.show()

print(f'AUC Score: {roc_auc:.4f}')

In [ ]:

# ============================================================
# 7.3 FEATURE IMPORTANCE (COEFFICIENTS)
# ============================================================

# Feature names for the 3 scaled inputs [Recency, Frequency, Monetary]
feature_names = ['Recency', 'Frequency', 'Monetary']

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': log_reg.coef_[0],
    'Abs_Coefficient': np.abs(log_reg.coef_[0])
}).sort_values('Abs_Coefficient', ascending=True)

fig = px.bar(
    feature_importance,
    x='Coefficient',
    y='Feature',
    orientation='h',
    color='Coefficient',
    color_continuous_scale='RdYlGn',
    title='Logistic Regression Feature Coefficients (Predicting High-Value Customer)',
    labels={'Coefficient': 'Coefficient Value', 'Feature': ''},
    text='Coefficient'
)
fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')
fig.update_layout(
    template='plotly_white',
    height=400,
    title_font_size=18
)
fig.show()

print('\nInterpretation:')
for _, row in feature_importance.sort_values('Abs_Coefficient', ascending=False).iterrows():
    direction = 'INCREASES' if row['Coefficient'] > 0 else 'DECREASES'
    print(f'  {row["Feature"]}: {direction} probability of being High-Value (coeff={row["Coefficient"]:.4f})')


---

## 8. Revenue Regression Analysis

We build a **Linear Regression model** to understand the relationship between customer behavioral features (Recency, Frequency) and their total monetary value. This helps in **revenue forecasting** at the customer level.

---

In [ ]:
# ============================================================
# 8. LINEAR REGRESSION - REVENUE PREDICTION
# ============================================================

# Use log-transform of Monetary for better linearity (revenue is right-skewed)
rfm['Log_Monetary'] = np.log1p(rfm['Monetary'])

# Features and target
X_reg = rfm[['Recency', 'Frequency']]
y_reg = rfm['Log_Monetary']

# Train-test split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Fit Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train_r, y_train_r)

# Predictions
y_pred_r = lin_reg.predict(X_test_r)

# Metrics
r2 = r2_score(y_test_r, y_pred_r)
mae = mean_absolute_error(y_test_r, y_pred_r)
rmse = np.sqrt(mean_squared_error(y_test_r, y_pred_r))

print('LINEAR REGRESSION RESULTS')
print('='*60)
print(f'R-squared (R2):             {r2:.4f}')
print(f'Mean Absolute Error (MAE):  {mae:.4f}')
print(f'Root Mean Sq Error (RMSE):  {rmse:.4f}')
print(f'\nCoefficients:')
for feat, coef in zip(X_reg.columns, lin_reg.coef_):
    print(f'  {feat}: {coef:.6f}')
print(f'  Intercept: {lin_reg.intercept_:.6f}')

In [ ]:
# ============================================================
# 8.1 ACTUAL vs PREDICTED PLOT
# ============================================================

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test_r,
    y=y_pred_r,
    mode='markers',
    marker=dict(color='#3498db', size=5, opacity=0.5),
    name='Predictions'
))

# Perfect prediction line
min_val = min(y_test_r.min(), y_pred_r.min())
max_val = max(y_test_r.max(), y_pred_r.max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(color='#e74c3c', width=2, dash='dash'),
    name='Perfect Prediction'
))

fig.update_layout(
    title=f'Actual vs Predicted Log(Monetary) | R2 = {r2:.4f}',
    title_font_size=18,
    xaxis_title='Actual Log(Monetary)',
    yaxis_title='Predicted Log(Monetary)',
    template='plotly_dark',
    height=550
)
fig.show()

In [ ]:
# ============================================================
# 8.2 RESIDUAL ANALYSIS
# ============================================================

residuals = y_test_r - y_pred_r

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Residuals vs Predicted', 'Residual Distribution')
)

fig.add_trace(
    go.Scatter(
        x=y_pred_r, y=residuals,
        mode='markers',
        marker=dict(color='#e74c3c', size=4, opacity=0.4),
        name='Residuals'
    ),
    row=1, col=1
)
fig.add_hline(y=0, line_dash='dash', line_color='white', row=1, col=1)

fig.add_trace(
    go.Histogram(
        x=residuals, nbinsx=40,
        marker_color='#3498db',
        name='Residual Distribution'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='Residual Analysis',
    title_font_size=18,
    template='plotly_dark',
    height=400,
    showlegend=False
)
fig.show()

#### Linear Regression - Business Interpretation

- **Frequency has a strong positive relationship with revenue**: Customers who buy more often spend more overall. This validates the importance of repeat-purchase programs.
- **Recency has a negative relationship**: The longer since a customer's last purchase, the lower their predicted revenue. This underscores the need for **re-engagement campaigns** for dormant customers.
- The R-squared value indicates the proportion of revenue variance explained by Recency and Frequency alone. Additional features (product preferences, geography, seasonality) would further improve predictions.

---

## 9. Business Interpretation & Conclusions

This section connects our analytical findings to **economic theory** and formulates **actionable business recommendations** for TATA's online retail leadership.

---

### 9.1 Pareto Principle (80/20 Rule)

In [ ]:
# ============================================================
# 9.1 PARETO PRINCIPLE ANALYSIS
# ============================================================

# Sort customers by revenue descending
customer_revenue = rfm[['CustomerID', 'Monetary']].sort_values('Monetary', ascending=False).reset_index(drop=True)
customer_revenue['CumulativeRevenue'] = customer_revenue['Monetary'].cumsum()
customer_revenue['CumulativeRevenue_Pct'] = customer_revenue['CumulativeRevenue'] / customer_revenue['Monetary'].sum() * 100
customer_revenue['Customer_Pct'] = (customer_revenue.index + 1) / len(customer_revenue) * 100

# Find what % of customers drive 80% of revenue
pct_at_80 = customer_revenue[customer_revenue['CumulativeRevenue_Pct'] >= 80]['Customer_Pct'].iloc[0]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=customer_revenue['Customer_Pct'],
    y=customer_revenue['CumulativeRevenue_Pct'],
    mode='lines',
    line=dict(color='#2ecc71', width=3),
    name='Cumulative Revenue',
    fill='tozeroy',
    fillcolor='rgba(46, 204, 113, 0.15)'
))

# Perfect equality line
fig.add_trace(go.Scatter(
    x=[0, 100], y=[0, 100],
    mode='lines',
    line=dict(color='gray', dash='dash'),
    name='Perfect Equality'
))

# 80% revenue line
fig.add_hline(y=80, line_dash='dot', line_color='#e74c3c',
              annotation_text='80% Revenue', annotation_position='top left')
fig.add_vline(x=pct_at_80, line_dash='dot', line_color='#e74c3c',
              annotation_text=f'{pct_at_80:.1f}% Customers', annotation_position='top right')

fig.update_layout(
    title=f'Pareto Analysis: {pct_at_80:.1f}% of Customers Drive 80% of Revenue',
    title_font_size=18,
    xaxis_title='% of Customers (ranked by revenue)',
    yaxis_title='% of Cumulative Revenue',
    template='plotly_dark',
    height=550
)
fig.show()

print(f'PARETO PRINCIPLE VALIDATION:')
print(f'  {pct_at_80:.1f}% of customers generate 80% of total revenue.')
print(f'  This closely follows the classic 80/20 Pareto distribution.')

### 9.2 Demand-Supply Analysis: Peak Demand Periods

In [ ]:
# ============================================================
# 9.2 DEMAND HEATMAP: DAY OF WEEK vs HOUR
# ============================================================

demand_heatmap = df.groupby(['DayName', 'Hour'])['TotalRevenue'].sum().reset_index()
demand_pivot = demand_heatmap.pivot_table(index='DayName', columns='Hour', values='TotalRevenue', fill_value=0)

# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
demand_pivot = demand_pivot.reindex(day_order)

fig = px.imshow(
    demand_pivot,
    color_continuous_scale='YlOrRd',
    title='Revenue Heatmap: Day of Week vs Hour of Day',
    labels=dict(x='Hour of Day', y='Day of Week', color='Revenue (GBP)'),
    aspect='auto'
)
fig.update_layout(
    template='plotly_dark',
    height=500,
    title_font_size=18
)
fig.show()

---

### 9.3 Comprehensive Business Conclusions

#### Economic Concepts Applied

| Economic Concept | Finding | Implication |
|-----------------|---------|-------------|
| **Pareto Principle** | A small fraction of customers generates the vast majority of revenue | Focus retention strategies on the top 20% high-value customers |
| **Demand Elasticity** | Revenue peaks during specific months (Q4), weekdays, and mid-day hours (10 AM - 2 PM) | Align marketing campaigns, staffing, and inventory with peak demand windows |
| **Market Concentration** | UK dominates revenue; Netherlands, EIRE, Germany are top international markets | Pursue focused international expansion in high-performing European markets |
| **Customer Lifetime Value** | Champions and Loyal Customer segments drive disproportionate revenue | Invest in loyalty programs and personalized offers for these segments |
| **Price Sensitivity** | Revenue is driven more by quantity than unit price (correlation analysis) | Volume-based promotions (bundles, multi-buy) may be more effective than discounting |
| **Churn Risk** | "At Risk" and "Lost" segments represent customers who were once active but have disengaged | Implement automated re-engagement campaigns (email, promotions) for these segments |

---

#### Actionable Business Recommendations

1. **Implement a Tiered Loyalty Program:** Based on RFM segments, offer differentiated benefits -- exclusive early access for Champions, milestone rewards for Loyal Customers, and win-back offers for At-Risk customers.

2. **Optimize Marketing Spend by Time:** Concentrate digital ad spending and email campaigns during peak engagement windows (weekday mornings, 10 AM - 2 PM). Reduce spend during off-peak hours and weekends (especially Saturday).

3. **Expand Strategically in Europe:** The Netherlands, Ireland, Germany, and France represent the strongest international markets. Invest in localized websites, local payment methods, and regional warehousing to reduce delivery times.

4. **Deploy Predictive Churn Models:** Use the Logistic Regression model as a foundation for identifying customers likely to churn. Trigger automated re-engagement sequences (discount codes, product recommendations) when a customer's predicted value drops.

5. **Seasonal Inventory Planning:** Q4 (September - November) shows the strongest revenue growth, likely driven by holiday purchasing. Pre-position inventory for top-selling products 6-8 weeks before the peak to avoid stockouts.

6. **Bundle and Cross-Sell Strategy:** Top-selling products should be combined into curated bundles. Use association analysis on customer purchase baskets to identify natural product pairings for cross-sell recommendations.

7. **Personalized Pricing & Promotions:** Rather than blanket discounts, use cluster-based targeting. High-frequency, low-monetary customers may respond to value bundles, while infrequent high-spenders may respond to premium product launches.

---

#### Summary of AI Techniques and Business Value

| Technique | Type | Business Value |
|-----------|------|----------------|
| **RFM Analysis** | Descriptive Analytics | Segments customers into actionable groups for targeted marketing |
| **K-Means Clustering** | Unsupervised ML | Discovers natural customer groupings without manual rules; validates and enriches RFM segments |
| **Logistic Regression** | Supervised ML (Classification) | Predicts high-value customers for proactive retention; identifies which features signal value |
| **Linear Regression** | Supervised ML (Regression) | Quantifies the relationship between customer behavior and revenue; supports forecasting |
| **Exploratory Data Analysis** | Descriptive Analytics | Reveals temporal, geographic, and product-level revenue patterns for operational planning |

> These AI-driven insights enable TATA's retail division to move from **intuition-based** to **evidence-based** decision-making, directly improving revenue efficiency and customer satisfaction.

---

## 10. Save Models & Outputs

We persist all trained models and key outputs to enable future deployment, scoring, and analysis without retraining.

In [ ]:
# ============================================================
# 10. SAVE MODELS & OUTPUTS
# ============================================================

# Create models directory
os.makedirs('models', exist_ok=True)

# Save K-Means model
joblib.dump(kmeans, 'models/kmeans_model.pkl')
print('Saved: models/kmeans_model.pkl')

# Save Logistic Regression model
joblib.dump(log_reg, 'models/logistic_regression_model.pkl')
print('Saved: models/logistic_regression_model.pkl')

# Save Linear Regression model
joblib.dump(lin_reg, 'models/linear_regression_model.pkl')
print('Saved: models/linear_regression_model.pkl')

# Save StandardScaler
joblib.dump(scaler, 'models/standard_scaler.pkl')
print('Saved: models/standard_scaler.pkl')

# Save RFM DataFrame
rfm.to_csv('models/rfm_analysis.csv', index=False)
print('Saved: models/rfm_analysis.csv')

print('\nAll models and outputs saved successfully.')
print(f'Files in models/ directory: {os.listdir("models")}')

---

### End of Analysis

**Project:** TATA Online Retail Store - Revenue Drivers Analysis  
**Group 11:** Payal Kunwar, Gaurav Kulkarni, Anshuman Atrey, Abdullah Haque, Shlok Vijay Kadam  
**Course:** Business Studies with Applied AI

---